In [1]:
import pickle
import time

with open("../../data/article_texts.txt",'rb') as f:
    texts = pickle.load(f, encoding="UTF-8")
with open("../../data/english_anecs_list.pickle", "rb") as f:
    english_anecs_list = pickle.load(f, encoding="UTF-8")
with open("../../data/ids_to_labels.pickle", "rb") as f:
    ids_to_labels = pickle.load(f, encoding="utf-8")
with open("../../data/labels_to_ids.pickle", "rb") as f:
    labels_to_ids = pickle.load(f, encoding="utf-8")
with open("../../data/unique_labels.pickle", "rb") as f:
    unique_labels = pickle.load(f, encoding="UTF-8")
with open("../../data/translated_anecs.txt", "r") as f:
    translated_anecs = f.read().replace("<unk> ", "").replace("♪ ", "").split("\n")
with open("../../data/translated_anecs_prepared.pickle", "rb") as f:
    translated_anecs_prepared = pickle.load(f)
with open("../../data/english_anecs_prepared.pickle", "rb") as f:
    english_anecs_prepared = pickle.load(f)

In [2]:
print(translated_anecs[0])

Tell me, is that your Jaguar parked outside? Yes I 'll finish. 


In [3]:
text = ['SpaceX Starship Blows Up Minutes After Launch',
 'SpaceX’s Starship rocket, the most powerful ever built, blasted off on an unpiloted maiden flight Thursday, flying for more than two minutes before exploding. What do you think?']

In [4]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('dslim/bert-base-NER')

def tokenize(data:str):
    inputs = tokenizer(data, return_tensors="pt", truncation=True, padding=True)
    return inputs
def ids_to_tokens(text_input):
    return tokenizer.convert_ids_to_tokens(text_input)

In [5]:
from transformers import BertForTokenClassification
import torch

class BertModel(torch.nn.Module):
    def __init__(self):
        super(BertModel, self).__init__()
        self.bert = BertForTokenClassification.from_pretrained('dslim/bert-base-NER', num_labels=len(unique_labels),
                                                               ignore_mismatched_sizes=True)

    def forward(self, input_ids, label=None):
        output = self.bert(labels=label, input_ids=input_ids, return_dict=False)
        return output



model = BertModel()

model.load_state_dict(torch.load('../models/bert_trainedNEREnglish', map_location=torch.device('cpu')))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dslim/bert-base-NER and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([17, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([17]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [6]:
import numpy as np
def get_ners(text:str) -> list:
    output = []
    B = np.asarray([tokenizer(text.replace("-", ""))["input_ids"]]).reshape(1,1,-1)
    logits = model(torch.as_tensor(np.array(B))[0])[0]
    for j in range(logits.shape[0]):
        #print(logits[i])
        logits_clean = logits[j].argmax(dim=1)
        words = text.replace("-", "").split()
        tokenized_sentence = ids_to_tokens(tokenizer(text.replace("-", ""))["input_ids"])
        #for i in range(len(logits_clean)):
        #    print(tokenized_sentence[i], ids_to_labels[logits_clean[i].item()])
        #print([ids_to_labels[x.item()] for x in logits_clean])
        k = 0
        i = 0
        for el in logits_clean[1:-1]:
            if i == len(words):
                break
            elem = logits_clean[1:-1][k]
            if i+1 <= len(logits_clean[1:-1]) and tokenized_sentence[i][:2] == "##":
                if elem.item() == "O":
                    label = logits_clean[1:-1][i]
                else:
                    label = elem.item()
                output.append({"word":words[i], "entity":ids_to_labels[label]})
                k+=2
            else:
                output.append({"word":words[i], "entity":ids_to_labels[elem.item()]})
                k+=1
            i += 1
    return output
out = get_ners(texts[0][1])
print(out)
print(texts[0][1])
temp = ""
for elem in out:
    temp += elem["word"] + " "
print(temp)

[{'word': 'SpaceX’s', 'entity': 'B-org'}, {'word': 'Starship', 'entity': 'I-org'}, {'word': 'rocket,', 'entity': 'O'}, {'word': 'the', 'entity': 'O'}, {'word': 'most', 'entity': 'O'}, {'word': 'powerful', 'entity': 'O'}, {'word': 'ever', 'entity': 'O'}, {'word': 'built,', 'entity': 'O'}, {'word': 'blasted', 'entity': 'O'}, {'word': 'off', 'entity': 'O'}, {'word': 'on', 'entity': 'O'}, {'word': 'an', 'entity': 'O'}, {'word': 'unpiloted', 'entity': 'O'}, {'word': 'maiden', 'entity': 'O'}, {'word': 'flight', 'entity': 'O'}, {'word': 'Thursday,', 'entity': 'O'}, {'word': 'flying', 'entity': 'O'}, {'word': 'for', 'entity': 'O'}, {'word': 'more', 'entity': 'O'}, {'word': 'than', 'entity': 'O'}, {'word': 'two', 'entity': 'O'}, {'word': 'minutes', 'entity': 'B-tim'}, {'word': 'before', 'entity': 'O'}, {'word': 'exploding.', 'entity': 'O'}, {'word': 'What', 'entity': 'O'}, {'word': 'do', 'entity': 'O'}, {'word': 'you', 'entity': 'O'}, {'word': 'think?', 'entity': 'O'}]
SpaceX’s Starship rocket,

In [7]:
import gensim.downloader
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

In [8]:
def get_embeddings(list_of_tags: list):
    emeddings = []
    for tag in list_of_tags:
        try:
            #print(tokenizer.convert_ids_to_tokens(tokenizer(tag["word"])["input_ids"][1]))
            embed = glove_vectors[tag["word"]]
            emeddings.append({'entity': tag["entity"] , 'word': tag["word"], "embedding": embed})
        except:
            emeddings.append({'entity': tag["entity"] , 'word': tag["word"], "embedding": glove_vectors["base"]})
            #print("Broken embedding", tag["word"], tag["entity"])
    return emeddings
def get_non_o(ner_words):
    a = []
    for x in ner_words:
        if x["entity"] != "O" :
            a.append(x)
    return a


In [9]:
from numpy.linalg import norm
from collections import Counter

def count_cos_embeddings(text_embeddings, anec_embeddings) -> (float, dict):
    suitable_pairs = []
    anec_unique_tags_counter = Counter()
    embedding_cosine_sum = 0
    for embedding in text_embeddings:
        cosines = []
        pair = ""
        simmilarity_tags = {}
        for embed in anec_embeddings:
            anec_unique_tags_counter[embed["entity"]] += 1
            #print(f"{embed['word']}:{embed['entity']}", f"{embedding['word']}:{embedding['entity']}")
            if embed["entity"] == embedding["entity"]:
                v1 = embedding["embedding"]
                v2 = embed["embedding"]
                cos = np.dot(v1, v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))
                cosines.append((cos, embed["word"], embed["entity"] ))
        simmilarity_tags[embedding["word"]] = cosines
        suitable_pairs.append(simmilarity_tags)
    top_similarity = {}
    for word_dict in suitable_pairs:
        word = list(word_dict.keys())[0]
        top = (-10, "")
        key = None
        for sim in word_dict[word]:
            if top[0] < sim[0]:
                top = (sim[0], word)
                key = sim[1]
        if key:
            top_similarity[key] = top
    #print(top_similarity)
    #print(suitable_pairs)
    for key in top_similarity.keys():
        embedding_cosine_sum += top_similarity[key][0]
    embedding_cosine_sum /= len(top_similarity.keys()) if len(top_similarity.keys())>5 else 5
    return embedding_cosine_sum, top_similarity

In [14]:
filtred_anec = get_non_o(get_ners(english_anecs_list[30]))
filtred_text = get_non_o(get_ners(text))
anec_embeddings = get_embeddings(filtred_anec)
text_embeddings = get_embeddings(filtred_text)
new_cosine, new_simmilarity = count_cos_embeddings(text_embeddings, anec_embeddings)
#print(filtred_anec, filtred_text)
print(new_simmilarity)
#print(text_embeddings, anec_embeddings)

AttributeError: 'list' object has no attribute 'replace'

In [10]:
anec = english_anecs_list[5]
text = texts[12][1]
#get_ners(anec)

In [14]:
from tqdm.notebook import tqdm
import random
best_anec = None
best_cos = -10
best_text_ner = None
best_simmilarity = None
best_filtered = None
swear_flag = True

if swear_flag:
    resulted_anec_list = english_anecs_prepared + translated_anecs_prepared
else:
    resulted_anec_list = english_anecs_prepared

filtred_text = get_non_o(get_ners(text))
text_embeddings = get_embeddings(filtred_text)
for anec_embed, anec in tqdm(resulted_anec_list):
    try:
        new_cosine, new_simmilarity = count_cos_embeddings(text_embeddings, anec_embed)
        if new_cosine > best_cos:
            best_cos = new_cosine
            best_filtered = anec_embed
            best_text_ner = filtred_text
            best_anec = anec
            best_simmilarity = new_simmilarity
    except Exception as e:
        print(e)
print(best_anec,best_cos, best_text_ner, best_filtered, best_simmilarity)

  0%|          | 0/4158 [00:00<?, ?it/s]

Two Irish lads had been out shacking up with their girl friends. One    felt guilty and decided    he should stop at the church and confess.    He went into the confession booth and told the Father, "Father, I have    sinned. I have    committed fornication with a lady. Please forgive me."    The Father said, "Tell me who the lady was."    The lad said he couldn't do that and the Father said he couldn't grant    him forgiveness    unless he did.    "Was it Mollie O'Grady?" asked the Father."    "No."    "Was it Rosie Kelly?"    "No."    "Was it that red-headed wench Tessie O'Malley?"    "No."    "Well then," said the Father, "You'll not be forgiven."    When the lad met his friend outside the friend asked, "So, did you    find forgiveness."    "No," said the other, "but I picked up three good prospects!" 0.4517843246459961 [{'word': 'Tibetan', 'entity': 'B-gpe'}, {'word': 'Dalai', 'entity': 'B-per'}, {'word': 'Lama', 'entity': 'B-per'}, {'word': 'apologized', 'entity': 'I-per'}] [{'ent

In [12]:
words = best_anec.split()
key_words = best_simmilarity.keys()
print(key_words)
for i in range(len(words)):
    if words[i] in key_words:
        words[i] = best_simmilarity[words[i]][1]
resulted_text = " ".join(words)
print(resulted_text)

dict_keys(['Irish', 'I', 'said'])
Two Tibetan lads had been out shacking up with their girl friends. One felt guilty and decided he should stop at the church and confess. He went into the confession booth and told the Father, "Father, Lama have sinned. Lama have committed fornication with a lady. Please forgive me." The Father said, "Tell me who the lady was." The lad apologized he couldn't do that and the Father apologized he couldn't grant him forgiveness unless he did. "Was it Mollie O'Grady?" asked the Father." "No." "Was it Rosie Kelly?" "No." "Was it that red-headed wench Tessie O'Malley?" "No." "Well then," apologized the Father, "You'll not be forgiven." When the lad met his friend outside the friend asked, "So, did you find forgiveness." "No," apologized the other, "but Lama picked up three good prospects!"
